## 结构化数组

假设我们要保存这样的数据：

||name|age|wgt|
|---|---|---|---|
|0|dan|1|23.1|
|1|ann|0|25.1|
|2|sam|2|8.3|

希望定义一个一维数组，每个元素有三个属性name,age,wgt,此时我们需要使用结构化数组。

In [1]:
import numpy as np

定义数组a：

|0|1|2|3|
|---|---|---|---|
|1.0|2.0|3.0|4.0|

In [2]:
a  = np.array([1.0, 2.0, 3.0, 4.0], np.float32)

使用view方法，将a对应的内存按照复数来解释：

In [4]:
a.view(np.complex64)

array([ 1.+2.j,  3.+4.j], dtype=complex64)


|0|1|2|3|
|---|---|---|---|
|1.0|2.0|3.0|4.0|
|real|imag|real|imag|

事实上，我们可以把复数看成一个结构体，第一部分是实部，第二部分是虚部，这样这个数组便可以看成是一个结构化的数组。

换句话说，我们只需要换种方式解释这段内存，便可以得到结构化的数组的效果！

|0|1|2|3|
|---|---|---|---|
|1.0|2.0|3.0|4.0|
|mass|vol|mass|vol|

例如，我们可以将第一个浮点数解释为质量，第二个浮点数解释为速度，则这段内存还可以看成是包含两个域（质量和速度）的结构体。

In [5]:
my_dtype = np.dtype([('mass','float32'),('vol','float32')])

In [6]:
a.view(my_dtype)

array([(1.0, 2.0), (3.0, 4.0)], 
      dtype=[('mass', '<f4'), ('vol', '<f4')])

这里，我们使用dtype创造了自定义的结构类型，然后用自定义的结构来解释数组a所占的内存。

这里f4表示四个字节浮点数，<表示小字节序。

利用这个自定义的结构类型，我们可以这样初始化结构数组：

In [7]:
my_data = np.array([(1,1), (2,1), (2,1), (1,3)],my_dtype)

print(my_data)

[(1.0, 1.0) (2.0, 1.0) (2.0, 1.0) (1.0, 3.0)]


第一个元素：

In [8]:
my_data[0]

(1.0, 1.0)

得到第一个元素的速度信息，可以使用域的名称来索引：

In [9]:
my_data[0]['vol']

1.0

得到所有的质量信息：

In [11]:
my_data['mass']

array([ 1.,  2.,  2.,  1.], dtype=float32)

自定义排序规则，先按速度，再按质量：

In [13]:
my_data.sort(order=('vol','mass'))

print(my_data)

[(1.0, 1.0) (2.0, 1.0) (2.0, 1.0) (1.0, 3.0)]


回到最初的例子，定义一个人的结构类型：

In [14]:
person_dtype = np.dtype([('name','S10'),('age','int'),('weight','float')])

查看类型所占字节数：

In [15]:
person_dtype.itemsize

26

产生一个3 x 4 共12人的空结构体数组：

In [16]:
people = np.empty((3,4), person_dtype)

分别赋值：

In [17]:
people['name'] = [['Brad', 'Jane', 'John', 'Fred'],
                  ['Henry', 'George', 'Brain', 'Amy'],
                  ['Ron', 'Susan', 'Jennife', 'Jill']]

In [18]:
people['age'] = [[33, 25, 47, 54],
                 [29, 61, 32, 27],
                 [19, 33, 18, 54]]

In [19]:
people['weight'] = [[135., 105., 255., 140.],
                    [154., 202., 137., 187.],
                    [188., 135., 88., 145.]]

In [20]:
print(people)

[[(b'Brad', 33, 135.0) (b'Jane', 25, 105.0) (b'John', 47, 255.0)
  (b'Fred', 54, 140.0)]
 [(b'Henry', 29, 154.0) (b'George', 61, 202.0) (b'Brain', 32, 137.0)
  (b'Amy', 27, 187.0)]
 [(b'Ron', 19, 188.0) (b'Susan', 33, 135.0) (b'Jennife', 18, 88.0)
  (b'Jill', 54, 145.0)]]


In [21]:
people[-1,-1]

(b'Jill', 54, 145.0)